In [16]:
from dataclasses import dataclass
from functools import reduce
import re


@dataclass(unsafe_hash=True)
class Coordinate:
    x: int
    y: int


@dataclass
class PotentialPart:
    part_number: int
    x_start: int
    x_end: int
    y: int

    surrounding_coordinates: list[Coordinate]

    def set_surrounding_coordinates(self):
        surrounding_x_start = self.x_start - 1
        surrounding_x_end = self.x_end + 1
        surrounding_y_above = self.y - 1
        surrounding_y_below = self.y + 1

        for y in range(surrounding_y_above, surrounding_y_below + 1):
            for x in range(surrounding_x_start, surrounding_x_end + 1):
                coord = Coordinate(x, y)
                self.surrounding_coordinates.append(coord)


@dataclass
class Symbol:
    coordinate: Coordinate
    symbol_char: str
    adjacent_part_nums: list[int]

    def update_adjacent_part_nums(self, part: PotentialPart):
        self.adjacent_part_nums.append(part.part_number)

# -------
# Parsing
# -------

def parse_line_into_potential_parts(
    line: str, line_y: int
) -> list[PotentialPart]:
    potential_parts = []

    number_matches = re.finditer(r"\d+", line)

    for match in number_matches:
        part_number = match.group(0)
        x_start = match.span(0)[0]
        x_end = (
            match.span(0)[1] - 1
        )  # re considers span end exclusive, which isn't what I want

        potential_part = PotentialPart(
            int(part_number), x_start, x_end, line_y, surrounding_coordinates=[]
        )
        potential_part.set_surrounding_coordinates()

        potential_parts.append(potential_part)

    return potential_parts


def parse_line_into_symbols(line: str, line_y: int) -> list[Symbol]:
    symbols = []

    # capture anything that is not a digit or period, aka symbols
    symbol_regex = r"(?!\d+|\.)(.)"
    symbol_matches = re.finditer(symbol_regex, line)

    for match in symbol_matches:
        symbol_coord = Coordinate(match.span(0)[0], line_y)
        symbol_char = match.group(0)

        symbol = Symbol(symbol_coord, symbol_char, [])

        symbols.append(symbol)

    return symbols


def parse_lines_into_potential_parts_and_symbols(
    lines: list[str],
) -> tuple[list[PotentialPart], list[Symbol]]:
    potential_parts = []
    symbols = []

    for y, line in enumerate(lines):
        line_potential_parts = parse_line_into_potential_parts(line, y)
        potential_parts.extend(line_potential_parts)

        line_symbols = parse_line_into_symbols(line, y)
        symbols.extend(line_symbols)

    return (potential_parts, symbols)


def create_symbols_map(symbols: list[Symbol]) -> dict[Coordinate, Symbol]:
    symbols_map: dict[Coordinate, Symbol] = {}

    for symbol in symbols:
        symbols_map[symbol.coordinate] = symbol

    return symbols_map

# ---------
# Filtering
# ---------

def filter_valid_parts(
    potential_parts: list[PotentialPart], symbols_map: dict[Coordinate, Symbol]
) -> list[PotentialPart]:
    valid_parts = []

    for part in potential_parts:
        for coord in part.surrounding_coordinates:
            adjacent_symbol = symbols_map.get(coord)
            if adjacent_symbol is not None:
                adjacent_symbol.update_adjacent_part_nums(part)
                valid_parts.append(part)
                break

    return valid_parts


def filter_gears_from_symbols(symbols: list[Symbol]) -> list[Symbol]:
    gears = [
        symbol
        for symbol in symbols
        if symbol.symbol_char == "*" and len(symbol.adjacent_part_nums) == 2
    ]

    return gears

# -----
# Other
# -----

def get_gear_ratio(gear: Symbol) -> int:
    gear_ratio = reduce(lambda x, y: x * y, gear.adjacent_part_nums)

    return gear_ratio

In [20]:
def main_part_1():
    with open("input.txt", "r") as file:
        lines = file.readlines()

    potential_parts, symbols = parse_lines_into_potential_parts_and_symbols(
        lines
    )
    symbols_map = create_symbols_map(symbols)
    valid_parts = filter_valid_parts(potential_parts, symbols_map)

    valid_part_nums = [part.part_number for part in valid_parts]
    result = sum(valid_part_nums)
    print(result)


def main_part_2():
    with open("input.txt", "r") as file:
        lines = file.readlines()

    potential_parts, symbols = parse_lines_into_potential_parts_and_symbols(
        lines
    )
    symbols_map = create_symbols_map(symbols)
    filter_valid_parts(potential_parts, symbols_map)

    gears = filter_gears_from_symbols(symbols)
    gear_ratios = [get_gear_ratio(gear) for gear in gears]

    result = sum(gear_ratios)
    print(result)


main_part_1()
main_part_2()

531932
73646890


In [18]:
# example validation

schematic = """
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
"""

ex_schematic = [line for line in schematic.split("\n") if line != ""]

ex_part_numbers = [467, 35, 633, 617, 592, 755, 664, 598]

# ---

potential_parts, symbols = parse_lines_into_potential_parts_and_symbols(
    ex_schematic
)


for potential_part in potential_parts:
    print(potential_part.part_number)

print()

for symbol in symbols:
    print(symbol.symbol_char)

print()

symbols_map = create_symbols_map(symbols)
for key, value in symbols_map.items():
    print(f"{key}: {value}")

print()

valid_parts = filter_valid_parts(potential_parts, symbols_map)
for valid_part in valid_parts:
    print(valid_part.part_number)

print()

for key, value in symbols_map.items():
    print(f"{key}: {value}")

print()

valid_part_nums = [part.part_number for part in valid_parts]
result = sum(valid_part_nums)
print(result)

print()

gears = filter_gears_from_symbols(symbols)
print(gears)

print()

gear_ratios = [get_gear_ratio(gear) for gear in gears]
print(gear_ratios)

print()

sum_gear_ratios = sum(gear_ratios)
print(sum_gear_ratios)

467
114
35
633
617
58
592
755
664
598

*
#
*
+
$
*

Coordinate(x=3, y=1): Symbol(coordinate=Coordinate(x=3, y=1), symbol_char='*', adjacent_part_nums=[])
Coordinate(x=6, y=3): Symbol(coordinate=Coordinate(x=6, y=3), symbol_char='#', adjacent_part_nums=[])
Coordinate(x=3, y=4): Symbol(coordinate=Coordinate(x=3, y=4), symbol_char='*', adjacent_part_nums=[])
Coordinate(x=5, y=5): Symbol(coordinate=Coordinate(x=5, y=5), symbol_char='+', adjacent_part_nums=[])
Coordinate(x=3, y=8): Symbol(coordinate=Coordinate(x=3, y=8), symbol_char='$', adjacent_part_nums=[])
Coordinate(x=5, y=8): Symbol(coordinate=Coordinate(x=5, y=8), symbol_char='*', adjacent_part_nums=[])

467
35
633
617
592
755
664
598

Coordinate(x=3, y=1): Symbol(coordinate=Coordinate(x=3, y=1), symbol_char='*', adjacent_part_nums=[467, 35])
Coordinate(x=6, y=3): Symbol(coordinate=Coordinate(x=6, y=3), symbol_char='#', adjacent_part_nums=[633])
Coordinate(x=3, y=4): Symbol(coordinate=Coordinate(x=3, y=4), symbol_char='*', adjacent_p